In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load and prepare data

Load the data

In [ ]:
df=pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')
df

Reverse the order of the data, some plottings will go better if the oldest dates are on top of the dataframe.

In [ ]:
df=df.iloc[::-1]

Also, stripping out the year and turning the date into a string will simplify plotting.

In [ ]:
df['Date']=df['date'].astype(str).str[5:]
df['DateTime']=pd.to_datetime(df['date'], format='%Y%m%d')

# Question 1:  How many states?

How many different "states" are there?

In [ ]:
# YOUR CODE HERE

# Question 2: Which 5 states have highest positive testing rate?

First we will make a new dataframe that has just the states, positives, negatives, and the positive fraction.

In [ ]:
# YOUR CODE HERE

Now find the 5 highest ratios.

In [ ]:
# YOUR CODE HERE

# Question 3:  Plot daily death toll for whole country

Let's plot daily death toll for the whole country.

In [ ]:
# YOUR CODE HERE

# Question 4:  Plot daily death toll for just Kansas and Missouri

Now let's focus on Missouri and Kansas, first we will graph their combined totals.

In [ ]:
# YOUR CODE HERE

Now let's separate KS and MO so that each has their own plot in the same graph.

In [ ]:
# YOUR CODE HERE

# Question 5:  Plot the daily testing rate for Kansas and Missouri

What is the daily testing rate looking like for Kansas and Missouri?

In [ ]:
# YOUR CODE HERE

# Question 6:  Make a table of peak testing day for each state

In [ ]:
# YOUR CODE HERE


# Question 8:  What is the peak death day for the whole country?

In [ ]:
# YOUR CODE HERE

# Challenge:  Plot the per capita testing fraction in descending order

First let's make a new dataframe that simply aggregates the total tests per state.

In [ ]:
# YOUR CODE HERE
tests_agged=

Now let's read in the population data, and keep only the parts we want.

In [ ]:
pop=pd.read_csv('http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv', usecols=['NAME', 'CENSUS2010POP'])

Now the problem is that our tests_agged has state abbreviations and our pop dataframe has full state names.  We need to convert one to the other before we can merge.

First let's load a file that has both.

In [ ]:
# RUN THIS CELL
abb=pd.read_csv('http://www.fonz.net/blog/wp-content/uploads/2008/04/states.csv')
abb

Merge the dataframe `tests_agged` with dataframe `abb`

In [ ]:
# YOUR CODE HERE

Now we can merge the `tests_agged` and `pop` dataframes because they both have a column with the full state name in them.

In [ ]:
# YOUR CODE HERE

Calculate the per 1000 tests, add to a column

In [ ]:
# YOUR CODE HERE

Sort, show the top 5.

In [ ]:
# YOUR CODE HERE

Create a seaborn barplot using the `tests_agged` dataset where:
- `data=tests_agged.sort_values(by='PerThousand', ascending=False)`
- `x='PerThousand'`
- `y='NAME'`



In [ ]:
plt.figure(figsize=[15,15])  # LEAVE THIS LINE
# YOUR CODE HERE
